In [14]:
import os
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define the repository URL to clone
repository_url = "https://github.com/benymaxparsa/Kotlin_projects_commit_diff"
# Clone the repository
!git clone {repository_url}

Cloning into 'Kotlin_projects_commit_diff'...
remote: Enumerating objects: 13557, done.
remote: Counting objects: 100% (13557/13557), done.
remote: Compressing objects: 100% (3415/3415), done.
remote: Total 13557 (delta 3014), reused 13555 (delta 3014), pack-reused 0
Receiving objects: 100% (13557/13557), 2.62 MiB | 9.32 MiB/s, done.
Resolving deltas: 100% (3014/3014), done.
Updating files: 100% (4437/4437), done.


In [3]:
# Get the repository name from the URL
repository_name = os.path.basename(repository_url).split(".")[0]
all_files = []
# Traverse the directory structure recursively
for root, dirs, files in os.walk(repository_name):
    for file in files:
        # Check if the file has a .kt or .java extension
        if file.endswith(".kt"):
            all_files.append(('kotlin','java',root, file))
#             runner(in_format='kotlin',out_format='java',file=file)
        elif file.endswith(".java"):
            all_files.append(('java','kotlin',root, file))
#             runner(in_format='java',out_format='kotlin',file=file)

In [31]:
repo_files = {}
for file in all_files:
  repo = file[2].split('/')[1]
  if repo not in repo_files:
    repo_files[repo]= []
  repo_files[repo].append(file)

repo_commits = {}
for repo in repo_files.keys():
  commits_count = len(set([file[2].split('/')[2].split('-')[-1] for file in repo_files[repo]]))
  repo_commits[repo] = commits_count

pd.DataFrame(columns=["Repository","Files","Commits"],data=list(zip(repo_files.keys(), # repo name
                                                                [len(repo) for repo in repo_files.values()], # files
                                                                repo_commits.values()))) # commits

,Repository,Files,Commits
0,picasso_Diff,234,20
1,architecture-samples_Diff,455,20
2,leakcanary_Diff,1074,20
3,BaseRecyclerViewAdapterHelper_Diff,206,20
4,sunflower_Diff,370,20
5,shadowsocks-android_Diff,405,20
6,iosched_Diff,839,19
7,okhttp_Diff,361,21
8,plaid_Diff,492,20


In [4]:
import re

# Define regex patterns for comments
java_single_line_comment_regex = r"\/\/.*"
java_multiline_comment_regex = r"\/\*(?:[^*]|\*(?!\/))*\*\/"
kotlin_single_line_comment_regex = r"\/\/.*"
kotlin_multiline_comment_regex = r"\/\*(?:[^*]|\*(?!\/))*\*\/"


def remove_comments(content,file_type):
    """
    Opens a Java or Kotlin file, removes comments, and saves the changes.

    Args:
        file_path: The path to the Java or Kotlin file.
    """


    # Determine file type based on extension
    if file_type == "java":
        pattern = java_single_line_comment_regex + "|" + java_multiline_comment_regex
    elif file_type == "kotlin":
        pattern = kotlin_single_line_comment_regex + "|" + kotlin_multiline_comment_regex
    else:
        raise ValueError(f"Unsupported file type: {file_type}")

    # Remove comments using regex
    clean_content = re.sub(pattern, "", content, )
    return clean_content
    # Write changes back to the file
    # with open(file_path, "w") as f:
    #     f.write(clean_content)

# Example usage
# file_path = "path/to/your/file.java"


In [32]:
response = ''
def runner(in_format,out_format, root, file):
    code = ''
    # writing original file next to the output to find it easily
    with open(os.path.join(root, file), "r",encoding="utf-8") as f:
      # Read the contents of the file
        code = f.readlines()
        code = ''.join(code)
    return remove_comments(code,in_format)

In [33]:
clean_codes = []
file_paths = []
for repo in repo_files.keys():
  for item in repo_files[repo]:
    if item[0] == 'kotlin':
      clean_code = runner(item[0],item[1],item[2],item[3])
      clean_codes.append(clean_code)
      file_paths.append(item[2]+'/'+item[3])
  clean_codes_str = '\n<code block>\n'.join(clean_codes)
  file_path_str = '\n'.join(file_paths)
  with open(f'./{repo}-merged.txt', "w",encoding="utf-8") as f:
    f.write(clean_codes_str)

  with open(f'./{repo}-paths.txt', "w",encoding="utf-8") as f:
    f.write(file_path_str)


In [40]:
# import zipfile
# text_files = [file for file in os.listdir("./") if file.endswith(".txt")]

In [42]:
# with zipfile.ZipFile('merged_paths_text.zip', 'w') as zipF:
#   for file in text_files:
#     zipF.write(file, compress_type=zipfile.ZIP_DEFLATED)